In [ ]:
import pandas as pd
import random
from pathlib import Path
import shutil
import subprocess
import sys

# ===== CONFIG =====
CSV_PATH = Path("labels.csv")          # je originele CSV
OUTPUT_DIR = Path("YOLO_subset")       # folder voor 50 afbeeldingen
SAMPLE_SIZE = 50                        # aantal afbeeldingen
LABELIMG_PATH = "labelImg"              # commando of pad naar labelImg exe
random.seed(42)
# ==================

# 1️⃣ Lees CSV en filter container-afbeeldingen
df = pd.read_csv(CSV_PATH)
df_containers = df[df['label'].isin([0, 1])]

if len(df_containers) < SAMPLE_SIZE:
    raise ValueError(f"Niet genoeg container-afbeeldingen in CSV! Alleen {len(df_containers)} beschikbaar.")

# 2️⃣ Shuffle en selecteer
df_sample = df_containers.sample(n=SAMPLE_SIZE, random_state=42)

# 3️⃣ Maak output folders aan
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
(IMAGES_DIR := OUTPUT_DIR / "images").mkdir(exist_ok=True)
(LABELS_DIR := OUTPUT_DIR / "labels").mkdir(exist_ok=True)

# 4️⃣ Kopieer afbeeldingen en maak lege YOLO-labels
for _, row in df_sample.iterrows():
    src_path = Path(row["image_path"])
    dest_img_path = IMAGES_DIR / src_path.name
    shutil.copy(src_path, dest_img_path)

    label_path = LABELS_DIR / (src_path.stem + ".txt")
    label_path.touch()  # leeg bestand

print(f"{SAMPLE_SIZE} afbeeldingen met container klaar in {OUTPUT_DIR}")

# 5️⃣ Open LabelImg op de folder
try:
    # Als labelImg in PATH staat
    subprocess.run([LABELIMG_PATH, str(IMAGES_DIR), str(LABELS_DIR)])
except FileNotFoundError:
    print(f"Kan LabelImg niet vinden. Zorg dat labelImg in PATH staat of pas LABELIMG_PATH aan.")


: 